In [35]:
from selenium import webdriver
from bs4 import BeautifulSoup as soup 
from webdriver_manager.chrome import ChromeDriverManager
d = webdriver.Chrome(ChromeDriverManager().install())

d.get('https://www.nba.com/stats/players/traditional/?sort=PTS&dir=-1')
s = soup(d.page_source, 'html.parser').find('table', {'class':'table'})

headers, [_, *data] = [i.text for i in s.find_all('th')], [[i.text for i in b.find_all('td')] for b in s.find_all('tr')]
final_data = [i for i in data if len(i) > 1]

final_data

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\sqq7\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [58]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd


# NBA season we will be analyzing
year = 1980

# URL page we will scraping (see image above)
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
# this is the HTML from the given URL
html = uReq(url)
page_soup = soup(html)

# use findALL() to get the column headers
page_soup.findAll('tr', limit=2)
# use getText()to extract the text we need into a list
headers = [th.getText() for th in page_soup.findAll('tr', limit=2)[0].findAll('th')]
# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]

# avoid the first header row
rows = page_soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

stats = pd.DataFrame(player_stats, columns = headers)

while year < 2020:
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    html = uReq(url)
    page_soup = soup(html)
    page_soup.findAll('tr', limit=2)
    headers = [th.getText() for th in page_soup.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]
    rows = page_soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
    
    stats_temp = pd.DataFrame(player_stats, columns = headers)
    stats = stats.append(stats_temp)
    year+=1
    

# Dropping Columns
stats.drop('Age',inplace=True,axis=1)
len(stats.index)


21480

21480

In [57]:
stats.iloc[20000]
# year
# take out ones that have nothing on it ex. 4000
# In season Trades

Player    Fred VanVleet
Pos                  PG
Tm                  TOR
G                    37
GS                    0
MP                  7.9
FG                  1.1
FGA                 3.0
FG%                .351
3P                  0.3
3PA                 0.8
3P%                .379
2P                  0.8
2PA                 2.2
2P%                .341
eFG%               .401
FT                  0.5
FTA                 0.6
FT%                .818
ORB                 0.1
DRB                 1.0
TRB                 1.1
AST                 0.9
STL                 0.4
BLK                 0.1
TOV                 0.4
PF                  1.0
PTS                 2.9
Name: 563, dtype: object